In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("geektech/flan-t5-large-lora-qa-gpt4")
model = AutoModelForSeq2SeqLM.from_pretrained("geektech/flan-t5-large-lora-qa-gpt4")
model.eval()

# Define the prompt template
def generate_relevance_prompt(question, document):
    return (
        f'''You are an evaluator tasked with assessing how relevant a document is to a user’s question. Your evaluation should consider the semantic alignment between the question and the document.

        Instructions:
        1. **Highly Relevant (Score: 0.5 to 1.0)**:
          - The document directly answers the question or fully aligns with its meaning.
          - Examples include direct answers, precise explanations, or highly related content.

        2. **Partially Relevant (Score: 0.0 to 0.5)**:
          - The document is somewhat related but misses key details or only vaguely addresses the question.

        3. **Not Relevant or Contradictory (Score: -1.0 to 0.0)**:
          - The document is unrelated, off-topic, or contradicts the question.

        Your output should only be a floating-point number between -1.0 and 1.0, based on the relevance criteria.

        ### Question:
        {question}

        ### Document:
        {document}

        ### Relevance Score:
        '''
    )

# Function to check relevance for multiple contexts
def check_relevance_for_all_contexts(question, contexts):
    results = []
    for context in contexts:
        prompt = generate_relevance_prompt(question, context)
        inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=512)

        with torch.no_grad():
            outputs = model.generate(inputs.input_ids, max_length=10)

        # Decode the model's output and parse the relevance score
        relevance_score_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        print(f"Raw Model Output: {relevance_score_text}")

        try:
            # Convert the text response to a numerical score
            relevance_score = float(relevance_score_text)
        except ValueError:
            # Handle cases where the model outputs textual classifications
            if relevance_score_text == "Highly Relevant":
                relevance_score = 1.0
            elif relevance_score_text == "Partially Relevant":
                relevance_score = 0.0
            elif relevance_score_text == "Not Relevant or Contradictory":
                relevance_score = -1.0
            else:
                relevance_score = 0.0  # Default for unexpected responses

        # Determine if the context is relevant based on the threshold
        is_relevant = relevance_score >= 0 # threshold
        results.append((context, relevance_score, is_relevant))

    return results

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

In [2]:
data = {
    "Historical Events": {
        "What year did World War II end?": [
            "World War II ended in 1945 after the surrender of Germany and Japan.",
            "World War I ended in 1918 with the signing of the Treaty of Versailles.",
            "The Cold War began after World War II and lasted for decades.",
            "Albert Einstein developed the theory of relativity in the early 20th century."
        ]
    },
    "Geography": {
        "What is the tallest mountain in the world?": [
            "Mount Everest, located in the Himalayas, is the tallest mountain in the world.",
            "The Nile is the longest river in the world, flowing through northeastern Africa.",
            "K2, also known as Mount Godwin-Austen, is the second-highest mountain in the world.",
            "The Grand Canyon is a popular tourist destination in the United States."
        ]
    },
    "Science": {
        "What is the chemical formula for water?": [
            "Water has the chemical formula H2O, consisting of two hydrogen atoms and one oxygen atom.",
            "Carbon dioxide (CO2) is a colorless gas found in Earth's atmosphere.",
            "Salt, commonly used in cooking, has the chemical formula NaCl.",
            "The periodic table organizes elements by their atomic number and properties."
        ]
    },
    "Literature": {
        "Who wrote 'Pride and Prejudice'?": [
            "'Pride and Prejudice' is a novel written by Jane Austen.",
            "'Moby Dick' was written by Herman Melville and tells the story of Captain Ahab.",
            "'To Kill a Mockingbird,' a classic novel, was written by Harper Lee.",
            "'1984' is a dystopian novel by George Orwell."
        ]
    },
    "Technology": {
        "What is the purpose of a URL?": [
            "A URL (Uniform Resource Locator) is used to specify the address of a web page on the internet.",
            "HTML (HyperText Markup Language) is used to structure web pages.",
            "An IP address identifies a device on a network but differs from a URL in its usage.",
            "The internet was invented to facilitate global communication and data sharing."
        ]
    },
    "Sports": {
        "Who won the FIFA World Cup in 2018?": [
            "France won the 2018 FIFA World Cup by defeating Croatia in the final.",
            "Germany won the 2014 FIFA World Cup, beating Argentina 1-0 in the final.",
            "Brazil holds the record for the most FIFA World Cup wins, with five titles.",
            "The UEFA Champions League is an annual soccer competition for European clubs."
        ]
    },
    "Current Events": {
        "Who is the current president of the United States?": [
            "Joe Biden is the current president of the United States, having taken office in January 2021.",
            "Donald Trump served as the 45th president of the United States from 2017 to 2021.",
            "Barack Obama, the 44th president of the United States, served two terms from 2009 to 2017.",
            "The United Nations is an international organization founded in 1945."
        ]
    },
    "Health": {
        "What is the primary function of the heart?": [
            "The heart pumps blood throughout the body, delivering oxygen and nutrients to tissues.",
            "The liver plays a key role in detoxifying chemicals and metabolizing drugs.",
            "The lungs are responsible for gas exchange, bringing oxygen into the blood and removing carbon dioxide.",
            "The brain controls bodily functions, processes sensory information, and enables thinking."
        ]
    }
}


In [3]:
for domain, questions in data.items():
    for question, contexts in questions.items():
        results = check_relevance_for_all_contexts(question, contexts)
        for context, relevance_score, is_relevant in results:
            print(f"Context: '{context}'\nRelevance Score: {relevance_score}\nIs Relevant: {is_relevant}\n")

Raw Model Output: 1.0
Raw Model Output: 1.0
Raw Model Output: 1.0
Raw Model Output: Not Relevant or Contradictory
Context: 'World War II ended in 1945 after the surrender of Germany and Japan.'
Relevance Score: 1.0
Is Relevant: True

Context: 'World War I ended in 1918 with the signing of the Treaty of Versailles.'
Relevance Score: 1.0
Is Relevant: True

Context: 'The Cold War began after World War II and lasted for decades.'
Relevance Score: 1.0
Is Relevant: True

Context: 'Albert Einstein developed the theory of relativity in the early 20th century.'
Relevance Score: -1.0
Is Relevant: False

Raw Model Output: 1.0
Raw Model Output: Not Relevant
Raw Model Output: 1.0
Raw Model Output: 1.0
Context: 'Mount Everest, located in the Himalayas, is the tallest mountain in the world.'
Relevance Score: 1.0
Is Relevant: True

Context: 'The Nile is the longest river in the world, flowing through northeastern Africa.'
Relevance Score: 0.0
Is Relevant: False

Context: 'K2, also known as Mount Godwi